In [1]:
import os

In [2]:
!git clone https://github.com/something-original/doctr

Cloning into 'doctr'...
remote: Enumerating objects: 12374, done.
remote: Counting objects: 100% (3393/3393), done.
remote: Compressing objects: 100% (960/960), done.
remote: Total 12374 (delta 2849), reused 2565 (delta 2422), pack-reused 8981 (from 1)
Receiving objects: 100% (12374/12374), 6.64 MiB | 11.84 MiB/s, done.
Resolving deltas: 100% (8896/8896), done.


In [3]:
os.chdir('doctr')

In [ ]:
!pip install -e . --upgrade
!pip install -r references/requirements.txt

In [ ]:
!python references/recognition/train_pytorch.py crnn_mobilenet_v3_small --train_path ../train --val_path ../val --vocab russian --epochs 10 --name doctr_v1 --freeze-backbone